In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from IPython.display import display,HTML
display(HTML("<style>.container  {width:100% !important;}</style>"))

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import finpy_tse as fpy
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import pearsonr
from scipy.linalg import cholesky
import os
import time
import matplotlib.colors as mcolors
from matplotlib.ticker import FuncFormatter
from jdatetime import datetime as jdatetime
import asyncio
from telegram import Bot
from telegram.error import TelegramError
import plotly.io as pio
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler

In [4]:
import os

# Path to the directory containing Excel files to clean
excel_dir = 'C:/Reinforcment_Learning_dataset/Excels_File_Periods/Output/'

def clean_excel_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(directory, filename)
            os.remove(file_path)
            print(f"Deleted file: {file_path}")

# Clean all Excel files in the directory
clean_excel_files(excel_dir)


Deleted file: C:/Reinforcment_Learning_dataset/Excels_File_Periods/Output/df1.xlsx
Deleted file: C:/Reinforcment_Learning_dataset/Excels_File_Periods/Output/khodro.xlsx
Deleted file: C:/Reinforcment_Learning_dataset/Excels_File_Periods/Output/MarketWatch 1403-04-26 22-51-11.xlsx
Deleted file: C:/Reinforcment_Learning_dataset/Excels_File_Periods/Output/OrderBook 1403-04-26 22-51-11.xlsx


In [5]:
PROJECT_ROOT_DIR = r"C:\Reinforcment_Learning_dataset"
CHAPTER_ID = "Output"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "Plot_Periods", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig, fig_id, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    print(path)
    
    # Convert Plotly figure to PNG image
    pio.write_image(fig, path, format=fig_extension, scale=resolution/100)

In [6]:
PROJECT_ROOT_DIR = "C:\Reinforcment_Learning_dataset"
CHAPTER_ID = "Output"
FILES_PATH = os.path.join(PROJECT_ROOT_DIR, "Excels_File_Periods", CHAPTER_ID)
os.makedirs(FILES_PATH, exist_ok=True)
def save_excel(data_frame, excel_filename):
    excel_path = os.path.join(FILES_PATH, excel_filename + ".xlsx")
    data_frame.to_excel(excel_path, index=True)
    print(f"Saving Excel file: {excel_path}")

In [7]:
PROJECT_ROOT_DIR = "C:\Reinforcment_Learning_dataset"
CHAPTER_ID = "Output"
FILES_PATH1 = os.path.join(PROJECT_ROOT_DIR, "Excels_File_Periods", CHAPTER_ID)
os.makedirs(FILES_PATH1, exist_ok=True)
def save_data(data_frame, excel_filename):
    excel_path = os.path.join(FILES_PATH1, excel_filename + ".xlsx")
    data_frame.to_excel(excel_path, index=True)
    print(f"Saving Data file: {excel_path}")

In [8]:
def fetch_stock_data(tickers, data_type):
    stock_data = {}
    for symbol in tickers:
        try:
            data = fpy.Get_Price_History(symbol,start_date=start_date,end_date=end_date,
                    ignore_date=False,adjust_price=True,show_weekday=False,double_date=False)
            #data.index.values = pd.to_datetime(data.index.values)
            #stock_data[symbol] = data[data_type]
            if data_type == 'Close':
                stock_data[symbol] = data['Close']
            elif data_type == 'Volume':
                stock_data[symbol] = data['Volume']
            elif data_type == 'Adj Close':
                stock_data[symbol] = data['Adj Close']
            elif data_type == 'Final':
                stock_data[symbol] = data['Final']
            else:
                print(f"Invalid data type: {data_type}. Please specify 'close_price' or 'volume'.")
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            pass
    return stock_data

In [9]:
def retry_get_market_watch(save_excel=True, save_path=FILES_PATH, retries=-1, delay=10):
    attempts = 0
    while retries == -1 or attempts < retries:
        try:
            ticks = fpy.Get_MarketWatch(save_excel=True, save_path=FILES_PATH)
            return ticks
        except Exception as e:
            error_message = str(e)
            print(f"Error fetching Market data: {error_message}. Retrying in {delay} seconds...")
            time.sleep(delay)
            attempts += 1
    print("Failed to fetch Market Watch after multiple retries.")
    return None

ticks = retry_get_market_watch(save_excel=True, save_path=FILES_PATH)
if ticks is not None:
    # Your further processing code here
    pass
else:
    print("Failed to fetch Market Watch. Please check your internet connection and try again.")

In [10]:
from datetime import datetime

directory = FILES_PATH

# Specify the part of the filename you want to search for
#file_part = input("Enter the part of the filename you want to search for: ")
file_part = 'Order'
# Search for files containing the specified part in their names
matching_files = [file for file in os.listdir(directory) if file_part in file]

# Dictionary to store modification times of matching Excel files
modification_times = {}

# Get modification times for each matching file
for file_name in matching_files:
    file_path = os.path.join(directory, file_name)
    if os.path.exists(file_path):
        modification_time = datetime.fromtimestamp(os.path.getmtime(file_path))
        modification_times[file_name] = modification_time

# Find the most recently modified file
if modification_times:
    last_file = max(modification_times, key=modification_times.get)
    last_file_path = os.path.join(directory, last_file)
    
    # Read the Excel file into a pandas DataFrame
    df = pd.read_excel(last_file_path,index_col=0)
    
    # Display the DataFrame
    print("DataFrame from the last saved file:")
    print(df)
    
    # Print the last modification time of the file
    print(f"Last modified time of the file '{last_file}': {modification_times[last_file]}")
else:
    print("No matching Excel files found in the directory.")

DataFrame from the last saved file:
         Day_LL   Day_UL  OB-Depth  Sell-No  Sell-Vol  Sell-Price  Buy-Price  \
Ticker                                                                         
آ س پ   17260.0  19460.0         1        1      7150       18500      17660   
NaN         NaN      NaN         2        1      8921       18570      17650   
NaN         NaN      NaN         3        1      2000       18590      17640   
NaN         NaN      NaN         4        2     20000       18690      17630   
NaN         NaN      NaN         5        2     12601       18700      17620   
...         ...      ...       ...      ...       ...         ...        ...   
یاقوت   21886.0  26748.0         1       10    407513       24371      24370   
NaN         NaN      NaN         2       62  28858383       24372      24369   
NaN         NaN      NaN         3        1   1000000       24373      24368   
NaN         NaN      NaN         4        1   1000000       24374      24367   
NaN 

In [11]:
directory = FILES_PATH

# Specify the part of the filename you want to search for
#file_part = input("Enter the part of the filename you want to search for: ")
file_part = 'Market'
# Search for files containing the specified part in their names
matching_files = [file for file in os.listdir(directory) if file_part in file]

# Dictionary to store modification times of matching Excel files
modification_times = {}

# Get modification times for each matching file
for file_name in matching_files:
    file_path = os.path.join(directory, file_name)
    if os.path.exists(file_path):
        modification_time = datetime.fromtimestamp(os.path.getmtime(file_path))
        modification_times[file_name] = modification_time

# Find the most recently modified file
if modification_times:
    last_file = max(modification_times, key=modification_times.get)
    last_file_path = os.path.join(directory, last_file)
    
    # Read the Excel file into a pandas DataFrame
    df1 = pd.read_excel(last_file_path,index_col=0)
    
    # Display the DataFrame
    print("DataFrame from the last saved file:")
    print(df1)
    
    # Print the last modification time of the file
    print(f"Last modified time of the file '{last_file}': {modification_times[last_file]}")
else:
    print("No matching Excel files found in the directory.")

DataFrame from the last saved file:
       Trade Type     Time   Open   High    Low  Close  Final  Close(%)  \
Ticker                                                                    
ورازی       تابلو  9:22:50   1550   1611   1550   1600   1598      2.17   
سپ          تابلو  9:19:04  13620  13620  13610  13610  14470     -5.94   
سکرد        تابلو  6:11:17      0      0      0   7960   7930      0.38   
های وب      تابلو  9:22:43    779    794    779    786    785      0.13   
زماهان      تابلو  9:23:01  27650  27650  27050  27650  27600      5.94   
...           ...      ...    ...    ...    ...    ...    ...       ...   
لوتوس       تابلو  9:21:52   2220   2239   2220   2239   2201      1.73   
وپترو       تابلو  9:22:30  37900  37900  37240  37900  37450      1.17   
پرداخت      تابلو  9:19:22   5800   5800   5700   5800   5930     -2.19   
بهیر        تابلو  9:18:03   4432   4469   4432   4469   4383      1.99   
دارا        تابلو  9:21:26  13052  13053  13052  13052  13052   

In [12]:
df.tail(3)

,Day_LL,Day_UL,OB-Depth,Sell-No,Sell-Vol,Sell-Price,Buy-Price,Buy-Vol,Buy-No,Download
Ticker,,,,,,,,,,
NaN,NaN,NaN,3,1,1000000,24373,24368,5000000,5,1403-04-26 22:53:32
NaN,NaN,NaN,4,1,1000000,24374,24367,5000000,5,1403-04-26 22:53:32
NaN,NaN,NaN,5,1,1000000,24375,24366,5000370,6,1403-04-26 22:53:32


In [13]:
df1.head()

,Trade Type,Time,Open,High,Low,Close,Final,Close(%),Final(%),Day_UL,...,No_Sell_R,No_Sell_I,Name,Market,Sector,Share-No,Base-Vol,Market Cap,EPS,Download
Ticker,,,,,,,,,,,,,,,,,,,,,
ورازی,تابلو,9:22:50,1550,1611,1550,1600,1598,2.17,2.04,1612,...,53.0,2.0,بیمه رازی,پایه,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,19039453000,1,30425045894000,-29.0,1403-04-26 22:53:32
سپ,تابلو,9:19:04,13620,13620,13610,13610,14470,-5.94,0.00,15330,...,2.0,0.0,پرداخت الکترونیک سامان کیش,بورس,رایانه و فعالیتهای وابسته به آن,13000000000,5200000,188110000000000,304.0,1403-04-26 22:53:32
سکرد,تابلو,6:11:17,0,0,0,7960,7930,0.38,0.00,8400,...,NaN,NaN,سیمان کردستان,بورس,سیمان، آهک و گچ,2200000000,1884422,17446000000000,749.0,1403-04-26 22:53:32
های وب,تابلو,9:22:43,779,794,779,786,785,0.13,0.00,832,...,29.0,0.0,داده گسترعصرنوین-های وب,بورس,اطلاعات و ارتباطات,60000000000,24000000,47100000000000,217.0,1403-04-26 22:53:32
زماهان,تابلو,9:23:01,27650,27650,27050,27650,27600,5.94,5.75,27650,...,112.0,1.0,مجتمع تولید گوشت مرغ ماهان,فرابورس,زراعت و خدمات وابسته,1130000000,1,31188000000000,400.0,1403-04-26 22:53:32


In [14]:
save_data(df1,'df1')

Saving Data file: C:\Reinforcment_Learning_dataset\Excels_File_Periods\Output\df1.xlsx


In [15]:
start_date = '1400-01-01'
end_date = '1403-12-29'
symbol = 'خودرو'

In [16]:
import finpy_tse as fpy 
dataset = fpy.Get_Price_History(symbol,start_date=start_date,end_date=end_date,
                    ignore_date=False,adjust_price=True,show_weekday=False,double_date=False)


In [17]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import pandas as pd
import time
import os
import google.auth.transport.requests

# Path to your credentials.json file
credentials_path = 'C:/Weights/credentials.json'

# Authenticate and create the PyDrive client
scopes = ['https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file(credentials_path, scopes=scopes)

# Create a request object to refresh credentials
request = google.auth.transport.requests.Request()
credentials.refresh(request)

gauth = GoogleAuth()
gauth.credentials = credentials
drive = GoogleDrive(gauth)

# Google Drive API service
service = build('drive', 'v3', credentials=credentials)

# Path to save the dataset locally
local_file_path = 'C:/Reinforcment_Learning_dataset/Excels_File_Periods/Output/df1.xlsx'

# Function to update the dataset in Google Drive
def update_dataset():
    # Example: Create a new dataset or load an existing one
    try:
        df = pd.read_excel(local_file_path)
    except FileNotFoundError:
        df = pd.DataFrame(columns=['timestamp', 'data'])

    # Example: Update the dataset (append new data)
    new_data = {'timestamp': pd.Timestamp.now(), 'data': 'new_value'}
    df = df.append(new_data, ignore_index=True)

    # Save the updated dataset locally
    df.to_excel(local_file_path, index=False)

    # Upload the updated dataset to Google Drive
    file_metadata = {'name': 'df1.xlsx'}
    media = MediaFileUpload(local_file_path, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

    # Check if the file already exists in Google Drive
    response = service.files().list(q="name='df1.xlsx'", spaces='drive').execute()
    if response['files']:
        file_id = response['files'][0]['id']
        # Update the existing file
        service.files().update(fileId=file_id, media_body=media).execute()
        print(f"Dataset 'df1.xlsx' updated and saved in Google Drive with file ID: {file_id}")
        # Print file details including owner information
        file_details = service.files().get(fileId=file_id, fields='name, parents, owners').execute()
        print(f"File Name: {file_details['name']}")
        print(f"Parent Folder ID: {file_details['parents'][0]}")
        print(f"Owner's Email: {file_details['owners'][0]['emailAddress']}")
    else:
        # Create a new file if it doesn't exist
        service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        print(f"New dataset 'df1.xlsx' created and saved in Google Drive.")

# Update the dataset
update_dataset()

Dataset 'df1.xlsx' updated and saved in Google Drive with file ID: 11LtIPi30frh2KhazUC2FEDfkzDXlot2Q
File Name: df1.xlsx
Parent Folder ID: 0AO1HCYyIPoz8Uk9PVA
Owner's Email: ronak-optics@algorithm-of-faraz-9188445973.iam.gserviceaccount.com


In [18]:
sadfasdf

NameError: name 'sadfasdf' is not defined

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import pandas as pd
import time
import os
import google.auth.transport.requests

# Path to your credentials.json file
credentials_path = 'C:/Weights/credentials.json'

# Authenticate and create the PyDrive client
scopes = ['https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file(credentials_path, scopes=scopes)

# Create a request object to refresh credentials
request = google.auth.transport.requests.Request()
credentials.refresh(request)

# Create PyDrive client
gauth = GoogleAuth()
gauth.credentials = credentials
drive = GoogleDrive(gauth)

# Google Drive API service
service = build('drive', 'v3', credentials=credentials)

# Path to save the dataset locally
local_file_path = 'C:/Reinforcment_Learning_dataset/Excels_File_Periods/Output/df1.xlsx'

# Function to update the dataset in Google Drive
def update_dataset():
    # Example: Load the existing dataset if it exists
    try:
        df = pd.read_excel(local_file_path)
    except FileNotFoundError:
        df = pd.DataFrame(columns=['timestamp', 'data'])

    # Example: Update the dataset (append new data)
    new_data = {'timestamp': pd.Timestamp.now(), 'data': 'new_value'}
    df = df.append(new_data, ignore_index=True)

    # Save the updated dataset locally
    df.to_excel(local_file_path, index=False)

    # Upload the updated dataset to Google Drive
    file_metadata = {'name': 'df1.xlsx', 'id': '11LtIPi30frh2KhazUC2FEDfkzDXlot2Q'}
    media = MediaFileUpload(local_file_path, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

    # Update the existing file in Google Drive
    service.files().update(fileId=file_metadata['id'], media_body=media).execute()
    print(f"Dataset 'df1.xlsx' updated and saved in Google Drive with file ID: {file_metadata['id']}")

# Update the dataset
update_dataset()


In [ ]:
dataset

In [ ]:
import time
from IPython.display import display, Javascript
from jdatetime import datetime as jdatetime

now = jdatetime.now()  # Get the current time

# Function to interrupt kernel, restart the notebook, and run all cells
def interrupt_restart_and_run_all():
    display(Javascript('''
        var kernel = Jupyter.notebook.kernel;
        kernel.interrupt();
        setTimeout(function() {
            kernel.restart();
            var run_all_cells = function() {
                var cells = Jupyter.notebook.get_cells();
                cells.forEach(function(cell, index) {
                    if (cell.cell_type == 'code') {
                        cell.execute();
                        cell.element.parents('.output_wrapper').find('.output').prepend('<div class="output_area"><div class="prompt output_prompt">Out[' + (index + 1) + ']:</div><div class="output_area"></div></div>');
                    }
                });
            };
            setTimeout(run_all_cells, 1000);
        }, 2000);
    '''))

# Interval for restart in seconds (30 minutes = 30 * 60 seconds)
restart_interval = 60

try:
    while True:
        # Print statements should be within parentheses
        print("It's between 8:30 AM and 3:00 PM. Continue to Running...")
        print("Interrupting kernel, restarting notebook, and running all cells every 30 minutes...")
        time.sleep(restart_interval)
        interrupt_restart_and_run_all()
except KeyboardInterrupt:
    print("Notebook restart and cell execution interrupted.")

In [ ]:
# Convert index values to strings and then filter numeric and non-numeric rows
numeric_df = df1[df1.index.astype(str).str.contains(r'\d')]
non_numeric_df = df1[~df1.index.astype(str).str.contains(r'\d')]

In [ ]:
non_numeric_df.tail(2)

In [ ]:
bors_names = non_numeric_df[non_numeric_df['Market'] == 'بورس']

In [ ]:
bors_names.tail(3)

In [ ]:
farabors_names = non_numeric_df[non_numeric_df['Market'] == 'فرابورس']

In [ ]:
farabors_names.tail(3)

In [ ]:
funds_names = non_numeric_df[non_numeric_df['Market'] == 'صندوق قابل معامله']

In [ ]:
funds_names.tail()

In [ ]:
keep_bors = [col for col in non_numeric_df.index if col in bors_names.index]

In [ ]:
bors_df = non_numeric_df[non_numeric_df.index.to_series().isin(keep_bors)]

In [ ]:
bors_df = bors_df.sort_index(axis=0)
bors_df.head(3)

In [ ]:
keep_farabors = [col for col in non_numeric_df.index if col in farabors_names.index]

In [ ]:
farabors_df = non_numeric_df[non_numeric_df.index.to_series().isin(keep_farabors)]

In [ ]:
farabors_df =farabors_df.sort_index(axis=0)
farabors_df.head(3)

In [ ]:
keep_funds = [col for col in non_numeric_df.index if col in funds_names.index]

In [ ]:
funds_df = non_numeric_df[non_numeric_df.index.to_series().isin(keep_funds)]

In [ ]:
funds_df =funds_df.sort_index(axis=0)
funds_df.head(3)

In [ ]:
# Assuming df is your DataFrame with Persian index labels
block_data = df1[df1.index.map(lambda x: x.endswith('2'))]

# Display the filtered DataFrame
block_data

In [ ]:
# Filter block_data to keep rows where the index ends with '2'
bors_block_2 = block_data[block_data.index.str.endswith('2')]

# Filter further based on the presence of index names (without '2') in keep_bors list
bors_block_data = bors_block_2[bors_block_2.index.str[:-1].isin(keep_bors)]

# Display the filtered DataFrame
bors_block_data.tail(2)

In [ ]:
farabors_block_2 = block_data[block_data.index.str.endswith('2')]

# Filter further based on the presence of index names (without '2') in keep_bors list
farabors_block_data = farabors_block_2[farabors_block_2.index.str[:-1].isin(keep_farabors)]

# Display the filtered DataFrame
farabors_block_data.tail(2)

In [ ]:
# Filter block_data to keep rows where the index ends with '2' or '4'
funds_block_2_4 = block_data[block_data.index.str.endswith(('2', '4'))]

# Filter further based on the presence of index names (without '2' and '4') in keep_bors list
funds_block_data = funds_block_2_4[funds_block_2_4.index.str[:-1].isin(keep_funds)]

# Display the filtered DataFrame
funds_block_data.tail(2)

In [ ]:
non_numeric_df.head(3)

In [ ]:
bors_df.columns

In [ ]:
input_bors_data = df[df.index.to_series().isin(keep_bors)]

In [ ]:
input_bors_data

In [ ]:
import plotly.graph_objects as go
import locale
from  datetime import datetime
# Set the locale to the desired format (e.g., 'en_US.UTF-8' for English)
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Assuming input_bors and output_bors are your input and output data
# Assuming you have bors_df DataFrame containing the data

# Calculate input and output
# Calculate input and output
input_bors = (input_bors_data['Buy-Price'] * input_bors_data['Buy-Vol']).sum()
output_bors = (input_bors_data['Sell-Price'] * input_bors_data['Sell-Vol']).sum()


# Format the numbers in accounting format with "Rials" suffix
input_bors_formatted = locale.format_string("%.0f Rials", input_bors, grouping=True)
output_bors_formatted = locale.format_string("%.0f Rials", output_bors, grouping=True)

# Create figure
fig = go.Figure()

# Add input bar trace
fig.add_trace(go.Bar(
    y=['ورود پول به بازار بورس'],
    x=[input_bors],
    orientation='h',
    text=[input_bors_formatted],  # Display the formatted value inside the bar
    name='ورود پول به بازار بورس',
    marker=dict(color='green'),  # Set color to green
    textfont=dict(size=20),  # Maximize font size
))

# Add output bar trace
fig.add_trace(go.Bar(
    y=['خروج پول از بازار بورس'],
    x=[output_bors],
    orientation='h',
    text=[output_bors_formatted],  # Display the formatted value inside the bar
    name='خروج پول از بازار بورس',
    marker=dict(color='red'),  # Set color to red
    textfont=dict(size=20),  # Maximize font size
))

# Update layout
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(
    title=f'ارزش سفارش های خرید و فروش بازار بورس(سبد گردان فراز)<br>Current Time: {now}',title_x=0.5,
    xaxis=dict(title='Money Flow'),
    yaxis=dict(title='Operation'),
    barmode='relative',
)
# Show plot
fig.show()
save_fig(fig,'input_output_money_bors')

In [ ]:
df1.columns

In [ ]:
# Define boolean conditions for buy and sell based on bors_df index
buy_condition = bors_df['Close'] >= bors_df['Day_UL']
sell_condition = (bors_df['Close'] <= bors_df['Day_LL'])

# Create a new column 'وضعیت' and initialize with empty strings
bors_df['وضعیت'] = ''

# Update 'وضعیت' based on conditions
bors_df.loc[buy_condition, 'وضعیت'] = 'صف خرید'
bors_df.loc[sell_condition, 'وضعیت'] = 'صف فروش'

# Filter bors_df based on the combined conditions for صف خرید and صف فروش
saf_bors_df = bors_df[buy_condition | sell_condition].copy()

# Calculate 'ارزش صف' based on 'وضعیت'
saf_bors_df.loc[saf_bors_df["وضعیت"] == 'صف خرید', 'ارزش صف'] = saf_bors_df['Day_UL'] * (saf_bors_df['Vol_Buy_R'] + saf_bors_df['Vol_Buy_I'])
saf_bors_df.loc[saf_bors_df["وضعیت"] == 'صف فروش', 'ارزش صف'] = saf_bors_df['Day_LL'] * (saf_bors_df['Vol_Sell_R'] + saf_bors_df['Vol_Sell_I'])

# Display the filtered DataFrame
saf_bors_df.tail(3)

In [ ]:
saf_bors_df.head()

In [ ]:
import plotly.graph_objects as go
from datetime import datetime

# Slice the DataFrame to show only the top 10 rows for each condition
top_10_buy = saf_bors_df[saf_bors_df["وضعیت"] == "صف خرید"].head(10)
top_10_sell = saf_bors_df[saf_bors_df["وضعیت"] == "صف فروش"].head(10)

# Reverse the order of the DataFrame
top_10_buy = top_10_buy[::-1].sort_values('ارزش صف')
top_10_sell = top_10_sell[::-1].sort_values('ارزش صف')

# Create Plotly figure
fig = go.Figure()

# Add horizontal bar chart for صف خرید (right side)
fig.add_trace(go.Bar(
    y=top_10_buy.index,
    x=top_10_buy['ارزش صف'],
    orientation='h',
    marker=dict(color='green'),  # Set color to green for صف خرید
    text=top_10_buy['ارزش صف'],  # Display the value on each bar
    textposition='inside',  # Display text inside each bar
    texttemplate='%{text:,.0f}  Rials',  # Text template for displaying the value with thousands separator
    insidetextanchor='middle',  # Anchor the text to the middle of the bar
    name='صف خرید',  # Label for صف خرید
))

# Add horizontal bar chart for صف فروش (left side)
fig.add_trace(go.Bar(
    y=top_10_sell.index,
    x=-top_10_sell['ارزش صف'],  # Negative values to mirror the bars
    orientation='h',
    marker=dict(color='red'),  # Set color to red for صف فروش
    text=top_10_sell['ارزش صف'],  # Display the value on each bar
    textposition='inside',  # Display text inside each bar
    texttemplate='%{text:,.0f}  Rials',  # Text template for displaying the value with thousands separator
    insidetextanchor='middle',  # Anchor the text to the middle of the bar
    name='صف فروش',  # Label for صف فروش
))

# Add title with current time
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(
    title=f' ارزش کل  صف های خرید و فروش بازار بورس(سبد گردان فراز)<br>Current Time: {now}',
    title_x=0.5,  # Center the title
    xaxis=dict(title='ارزش صف'),
    yaxis=dict(title='Index'),
    barmode='relative',  # Stacked bars
)

# Show the plot
fig.show()
save_fig(fig,'Saf_bors')

In [ ]:
bors_df.columns

In [ ]:
#bors_df['Total_Values'] = bors_df['Final'] * (bors_df['Vol_Buy_I'] + bors_df['Vol_Buy_R'])
arzesh_bors = bors_df.sort_values('Value')

In [ ]:
arzesh_bors = arzesh_bors.dropna(axis=0)

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Slice the DataFrame to include only the top 20 rows
arzesh_bors_top_20 = arzesh_bors.iloc[-20:]

# Create Plotly figure
fig = go.Figure()

# Add horizontal bar chart
fig.add_trace(go.Bar(
    y=arzesh_bors_top_20.index,
    x=arzesh_bors_top_20['Value'],
    orientation='h',
    marker=dict(color=np.log(arzesh_bors_top_20['Value']), colorscale='Viridis', colorbar=dict(title='Bors Values')),
    text=arzesh_bors_top_20['Value'],  # Display the volume value on each bar
    textposition='inside',  # Display text inside each bar
    texttemplate='%{text}',  # Text template for displaying the value
    insidetextanchor='middle',  # Anchor the text to the middle of the bar
    width=0.4  # Adjust the width of the bars
))

# Update layout
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(
    title=f'بیست سهم بازار بورس با بیشترین ارزش معاملات (سبد گردان فراز) <br>Current Time: {now}',
    title_x=0.5,
    xaxis=dict(title='Arzesh'),
    yaxis=dict(title='Ticker'),
)

# Show the plot
fig.show()
save_fig(fig, 'Arzesh_bors')

In [ ]:
input_farabors_data = df[df.index.to_series().isin(keep_farabors)]

In [ ]:
import plotly.graph_objects as go
import locale
from datetime import datetime

# Set the locale to the desired format (e.g., 'en_US.UTF-8' for English)
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Assuming input_bors and output_bors are your input and output data
# Assuming you have farabors_df DataFrame containing the data

# Calculate input and output
input_bors = (input_farabors_data['Buy-Price'] * input_farabors_data['Buy-Vol']).sum()
output_bors = (input_farabors_data['Sell-Price'] * input_farabors_data['Sell-Vol']).sum()


# Format the numbers in accounting format with "Rials" suffix
input_bors_formatted = locale.format_string("%.0f Rials", input_bors, grouping=True)
output_bors_formatted = locale.format_string("%.0f Rials", output_bors, grouping=True)

# Create figure
fig = go.Figure()

# Add input bar trace
fig.add_trace(go.Bar(
    y=['ورود پول به بازار فرابورس'],
    x=[input_bors],
    orientation='h',
    text=[input_bors_formatted],  # Display the formatted value inside the bar
    name='ورود پول به بازار فرابورس',
    marker=dict(color='green'),  # Set color to green
    textfont=dict(size=20),  # Maximize font size
))

# Add output bar trace
fig.add_trace(go.Bar(
    y=['خروج پول از بازار فرابورس'],
    x=[output_bors],
    orientation='h',
    text=[output_bors_formatted],  # Display the formatted value inside the bar
    name='خروج پول از بازار فرابورس',
    marker=dict(color='red'),  # Set color to red
    textfont=dict(size=20),  # Maximize font size
))

# Update layout
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(
    title=f'ارزش سفارش های خرید و فروش بازار فرابورس(سبد گردان فراز)<br>Current Time: {now}', title_x=0.5,
    xaxis=dict(title='Money Flow'),
    yaxis=dict(title='Operation'),
    barmode='relative',
)

# Show plot
fig.show()
save_fig(fig,'input_output_money_farabors')

In [ ]:
# Define boolean conditions for buy and sell based on farabors_df index
buy_condition = farabors_df['Close'] >= farabors_df['Day_UL']
sell_condition = farabors_df['Close'] <= farabors_df['Day_LL']

# Create a new column 'وضعیت' and initialize with empty strings
farabors_df['وضعیت'] = ''

# Update 'وضعیت' based on conditions
farabors_df.loc[buy_condition, 'وضعیت'] = 'صف خرید'
farabors_df.loc[sell_condition, 'وضعیت'] = 'صف فروش'

# Filter farabors_df based on the combined conditions for صف خرید and صف فروش
saf_farabors_df = farabors_df[buy_condition | sell_condition].copy()

# Calculate 'ارزش صف' based on 'وضعیت'
saf_farabors_df.loc[saf_farabors_df["وضعیت"] == 'صف خرید', 'ارزش صف'] = saf_farabors_df['Day_UL'] * (saf_farabors_df['Vol_Buy_R'] + saf_farabors_df['Vol_Buy_I'])
saf_farabors_df.loc[saf_farabors_df["وضعیت"] == 'صف فروش', 'ارزش صف'] = saf_farabors_df['Day_LL'] * (saf_farabors_df['Vol_Sell_R'] + saf_farabors_df['Vol_Sell_I'])

# Display the filtered DataFrame
saf_farabors_df

In [ ]:
import plotly.graph_objects as go
from datetime import datetime

# Slice the DataFrame to show only the top 10 rows for each condition
top_10_buy = saf_farabors_df[saf_farabors_df["وضعیت"] == "صف خرید"].head(10)
top_10_sell = saf_farabors_df[saf_farabors_df["وضعیت"] == "صف فروش"].head(10)

# Reverse the order of the DataFrame
top_10_buy = top_10_buy[::-1].sort_values('ارزش صف')
top_10_sell = top_10_sell[::-1].sort_values('ارزش صف')

# Create Plotly figure
fig = go.Figure()

# Add horizontal bar chart for صف خرید (right side)
fig.add_trace(go.Bar(
    y=top_10_buy.index,
    x=top_10_buy['ارزش صف'],
    orientation='h',
    marker=dict(color='green'),  # Set color to green for صف خرید
    text=top_10_buy['ارزش صف'],  # Display the value on each bar
    textposition='inside',  # Display text inside each bar
    texttemplate='%{text:,.0f}  Rials',  # Text template for displaying the value with thousands separator
    insidetextanchor='middle',  # Anchor the text to the middle of the bar
    name='صف خرید',  # Label for صف خرید
))

# Add horizontal bar chart for صف فروش (left side)
fig.add_trace(go.Bar(
    y=top_10_sell.index,
    x=-top_10_sell['ارزش صف'],  # Negative values to mirror the bars
    orientation='h',
    marker=dict(color='red'),  # Set color to red for صف فروش
    text=top_10_sell['ارزش صف'],  # Display the value on each bar
    textposition='inside',  # Display text inside each bar
    texttemplate='%{text:,.0f}  Rials',  # Text template for displaying the value with thousands separator
    insidetextanchor='middle',  # Anchor the text to the middle of the bar
    name='صف فروش',  # Label for صف فروش
))

# Add title with current time
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(
    title=f'ارزش کل  صف های خرید و فروش بازار فرابورس (سبد گردان فراز)<br>Current Time: {now}',
    title_x=0.5,  # Center the title
    xaxis=dict(title='ارزش صف'),
    yaxis=dict(title='Index'),
    barmode='relative',  # Stacked bars
)

# Show the plot
fig.show()
save_fig(fig,'Saf_farabors')

In [ ]:
farabors_df.columns

In [ ]:
arzesh_farabors = farabors_df.sort_values('Value')

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Slice the DataFrame to include only the top 20 rows
arzesh_farabors_top_20 = arzesh_farabors.iloc[-20:]

# Create Plotly figure
fig = go.Figure()

# Add horizontal bar chart
fig.add_trace(go.Bar(
    y=arzesh_farabors_top_20.index,
    x=arzesh_farabors_top_20['Value'],
    orientation='h',
    marker=dict(color=np.log(arzesh_farabors_top_20['Value']), colorscale='Viridis', colorbar=dict(title='Farabors Values')),
    text=arzesh_farabors_top_20['Value'],  # Display the volume value on each bar
    textposition='inside',  # Display text inside each bar
    texttemplate='%{text}',  # Text template for displaying the value
    insidetextanchor='middle',  # Anchor the text to the middle of the bar
    width=0.4  # Adjust the width of the bars
))

# Update layout
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(
    title=f'بیست سهم بازار فرابورس با بیشترین ارزش معاملات (سبد گردان فراز) <br>Current Time: {now}',
    title_x=0.5,
    xaxis=dict(title='Arzesh'),
    yaxis=dict(title='Ticker'),
)

# Show the plot
fig.show()
save_fig(fig, 'Arzesh_farabors')

In [ ]:
input_funds_data = df[df.index.to_series().isin(keep_funds)]

In [ ]:
import plotly.graph_objects as go
import locale
from datetime import datetime

# Set the locale to the desired format (e.g., 'en_US.UTF-8' for English)
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Assuming input_bors and output_bors are your input and output data
# Assuming you have funds_df DataFrame containing the data

# Calculate input and output
input_bors = (input_funds_data['Buy-Price'] * input_funds_data['Buy-Vol']).sum()
output_bors = (input_funds_data['Sell-Price'] * input_funds_data['Sell-Vol']).sum()


# Format the numbers in accounting format with "Rials" suffix
input_bors_formatted = locale.format_string("%.0f Rials", input_bors, grouping=True)
output_bors_formatted = locale.format_string("%.0f Rials", output_bors, grouping=True)

# Create figure
fig = go.Figure()

# Add input bar trace
fig.add_trace(go.Bar(
    y=['ورود پول به صندوق ها '],
    x=[input_bors],
    orientation='h',
    text=[input_bors_formatted],  # Display the formatted value inside the bar
    name='ورود پول به صندوق ها',
    marker=dict(color='green'),  # Set color to green
    textfont=dict(size=20),  # Maximize font size
))

# Add output bar trace
fig.add_trace(go.Bar(
    y=['خروج پول از صندوق ها'],
    x=[output_bors],
    orientation='h',
    text=[output_bors_formatted],  # Display the formatted value inside the bar
    name='خروج پول از صندوق ها',
    marker=dict(color='red'),  # Set color to red
    textfont=dict(size=20),  # Maximize font size
))

# Update layout
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(
    title=f'ارزش سفارش های خرید و فروش صندوق ها (سبد گردان فراز)<br>Current Time: {now}',
    title_x=0.5,
    xaxis=dict(title='Money Flow'),
    yaxis=dict(title='Operation'),
    barmode='relative',
)

# Show plot
fig.show()
save_fig(fig,'input_output_money_funds')

In [ ]:
arzesh_funds = funds_df.sort_values('Value')

In [ ]:
import plotly.graph_objects as go
import numpy as np
from datetime import datetime

# Slice the DataFrame to include only the top 20 rows
arzesh_funds_top_20 = arzesh_funds.iloc[-20:]

# Create Plotly figure
fig = go.Figure()

# Add horizontal bar chart
fig.add_trace(go.Bar(
    y=arzesh_funds_top_20.index,
    x=arzesh_funds_top_20['Value'],
    orientation='h',
    marker=dict(color=np.log(arzesh_funds_top_20['Value']), colorscale='Jet', colorbar=dict(title='Funds Values')),
    text=arzesh_funds_top_20['Value'],  # Display the volume value on each bar
    textposition='inside',  # Display text inside each bar
    texttemplate='%{text}',  # Text template for displaying the value
    insidetextanchor='middle',  # Anchor the text to the middle of the bar
    width=0.4  # Adjust the width of the bars
))

# Update layout
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(
    title=f'بیست صندوق با بیشترین ارزش معاملاتی (سبد گردان فراز) <br>Current Time: {now}',
    title_x=0.5,
    xaxis=dict(title='Arzesh'),
    yaxis=dict(title='Ticker'),
)

# Show the plot
fig.show()
save_fig(fig,'Arzesh_funds')

In [ ]:
arzesh_bors_block_data = bors_block_data.sort_values('Value')

In [ ]:
import plotly.graph_objects as go
import numpy as np
from datetime import datetime

# Slice the DataFrame to include only the top 20 rows
arzesh_bors_block_data_top_20 = arzesh_bors_block_data.iloc[-20:]

# Create Plotly figure
fig = go.Figure()

# Add horizontal bar chart
fig.add_trace(go.Bar(
    y=arzesh_bors_block_data_top_20.index,
    x=arzesh_bors_block_data_top_20['Value'],
    orientation='h',
    marker=dict(color=np.log(arzesh_bors_block_data_top_20['Value']), colorscale='Oranges', colorbar=dict(title='Bors Block Value')),
    text=arzesh_bors_block_data_top_20['Value'],  # Display the volume value on each bar
    textposition='inside',  # Display text inside each bar
    texttemplate='%{text}',  # Text template for displaying the value
    insidetextanchor='middle',  # Anchor the text to the middle of the bar
    width=0.4  # Adjust the width of the bars
))

# Update layout
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(
    title=f'بیست سهم بلوکی بازار بورس با بیشترین ارزش معاملاتی (سبد گردان فراز) <br>Current Time: {now}',
    title_x=0.5,
    xaxis=dict(title='Arzesh'),
    yaxis=dict(title='Ticker'),
)

# Show the plot
fig.show()
save_fig(fig,'Arzesh_block_bors')

In [ ]:
arzesh_farabors_block_data = farabors_block_data.sort_values('Value')

In [ ]:
import plotly.graph_objects as go
import numpy as np
from datetime import datetime

# Slice the DataFrame to include only the top 20 rows
arzesh_farabors_block_data_top_20 = arzesh_farabors_block_data.iloc[-20:]

# Create Plotly figure
fig = go.Figure()

# Add horizontal bar chart
fig.add_trace(go.Bar(
    y=arzesh_farabors_block_data_top_20.index,
    x=arzesh_farabors_block_data_top_20['Value'],
    orientation='h',
    marker=dict(color=np.log(arzesh_farabors_block_data_top_20['Value']), colorscale='HSV', colorbar=dict(title='Farabors Block Value')),
    text=arzesh_farabors_block_data_top_20['Value'],  # Display the volume value on each bar
    textposition='inside',  # Display text inside each bar
    texttemplate='%{text}',  # Text template for displaying the value
    insidetextanchor='middle',  # Anchor the text to the middle of the bar
    width=0.4  # Adjust the width of the bars
))

# Update layout
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(
    title=f'بیست سهم بلوکی بازار فرابورس با بیشترین ارزش معاملاتی (سبد گردان فراز) <br>Current Time: {now}',
    title_x=0.5,
    xaxis=dict(title='Arzesh'),
    yaxis=dict(title='Ticker'),
)

# Show the plot
fig.show()
save_fig(fig, 'Arzesh_block_farabors')

In [ ]:
arzesh_funds_block_data = funds_block_data.sort_values('Value')

In [ ]:
import plotly.graph_objects as go
import numpy as np
from datetime import datetime

# Slice the DataFrame to include only the top 20 rows
arzesh_funds_block_data_top_20 = arzesh_funds_block_data.iloc[-20:]

# Create Plotly figure
fig = go.Figure()

# Add horizontal bar chart
fig.add_trace(go.Bar(
    y=arzesh_funds_block_data_top_20.index,
    x=arzesh_funds_block_data_top_20['Value'],
    orientation='h',
    marker=dict(color=np.log(arzesh_funds_block_data_top_20['Value']), colorscale='HSV', colorbar=dict(title='Funds Block Value')),
    text=arzesh_funds_block_data_top_20['Value'],  # Display the volume value on each bar
    textposition='inside',  # Display text inside each bar
    texttemplate='%{text}',  # Text template for displaying the value
    insidetextanchor='middle',  # Anchor the text to the middle of the bar
    width=0.4  # Adjust the width of the bars
))

# Update layout
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(
    title=f'بیست بلوک روی صندوق ها با بیشترین ارزش معاملاتی (سبد گردان فراز) <br>Current Time: {now}',
    title_x=0.5,
    xaxis=dict(title='Arzesh'),
    yaxis=dict(title='Ticker'),
)

# Show the plot
fig.show()
save_fig(fig,'Arzesh_block_funds')

In [ ]:
arzesh_total_market = df1['Value'].sum()
last_price_total_market = df1['Open'].sum()
today_price_total_market = df1['Final'].sum()
diff_prices_total_market = today_price_total_market - last_price_total_market

In [ ]:
import plotly.graph_objects as go
import locale
from datetime import datetime

# Set the locale to the desired format (e.g., 'en_US.UTF-8' for English)
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Calculate the percentage change
percentage_change = (diff_prices_total_market / last_price_total_market) * 100

# Create a Plotly figure
fig = go.Figure()

# Add vertical bars for market values and percentage change
fig.add_trace(go.Bar(
    x=['ارزش ریالی کل بازار', 'درصد تغییرات قیمت باز شدن به پایانی'],
    y=[arzesh_total_market, 0],  # Start the percentage change bar from 0
    marker=dict(color=['blue', 'orange']),
    text=[locale.format_string("%.0f Rials", arzesh_total_market, grouping=True), f'{percentage_change:.2f}%'],
    textposition='outside',
))

# Add another bar trace for the percentage change
fig.add_trace(go.Bar(
    x=['درصد تغییرات'],
    y=[percentage_change],
    marker=dict(color='orange'),  # Use orange color for the percentage change bar
    text=f'{percentage_change:.2f}%',
    textposition='outside',
))

# Update layout
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(
    title=f'گزارش ارزش ریالی بازار و درصد تغییرات نسبت به روز قبل (سبد گردان فراز) <br>Current Time: {now}',
    title_x=0.5,
    xaxis=dict(title='Category'),
    yaxis=dict(title='Value'),
    barmode='group',  # Display bars side by side
)

# Show the plot
fig.show()
save_fig(fig,'Total_market_value')

In [ ]:
current_dir = %pwd
print("Current working directory:", current_dir)

# Construct the full path of the notebook file
notebook_filename = "Optic_to_OptionQuant_channel_version4.ipynb"  # Replace with your notebook filename
notebook_path = os.path.abspath(os.path.join(current_dir, notebook_filename))
print("Notebook path:", notebook_path)

In [ ]:
import yfinance as yf
# Define the ticker symbols for commodities along with their Persian names
ticker_symbols = {
    'ALI=F': 'آلومینیوم',
    'BTC-USD': 'بیت‌کوین',
    'BZ=F': 'نفت-برنت',
    'CL=F': 'نفت-غرب-تگزاس',
    'GC=F': 'طلا',
    'HG=F': 'مس',
    'HRC=F': 'ورق-فولاد',
    'KC=F': 'قهوه',
    'NG=F': 'گاز-طبیعی',
    'SB=F': 'شکر',
    'ZC=F': 'ذرت',
    'ZW=F': 'گندم'
}

# Initialize an empty list to store the data for the table
table_data = []

# Iterate over each ticker symbol and fetch its data
for ticker_symbol, persian_name in ticker_symbols.items():
    # Get historical data
    data = yf.download(ticker_symbol, period='1mo')
    
    # Drop NaN values
    data.dropna(inplace=True)
    
    # Get last close price and round it
    last_close_price = round(data['Close'].iloc[-1], 2)
    
    # Append data to the table
    table_data.append([persian_name, last_close_price])

# Create a DataFrame for the table
columns = ['نام کالا', 'آخرین قیمت']
table_df = pd.DataFrame(table_data, columns=columns)

# Create table trace
table_trace = go.Table(
    header=dict(values=['نام کالا', 'آخرین قیمت'],
                fill_color='darkblue',
                font=dict(color='white')),
    cells=dict(values=[table_df['نام کالا'], table_df['آخرین قیمت']],
               fill_color='orange',
               font=dict(color='black'))
)

# Create figure
fig = go.Figure(data=table_trace)

# Update layout
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
fig.update_layout(title=f'بازارهای جهانی در یک نگاه (سبد گردان فراز) <br>Current Time: {now}',title_x=0.5)#, margin=dict(l=0, r=0, t=0, b=0))

# Show the figure
save_fig(fig, 'Last_Price_of_CME_Commodity')
fig.show()

import speedtest
import time

def measure_speed_with_retry(st, max_retries=10, delay=5):
    """
    Measure internet speed with retries and delay.
    
    Parameters:
        st (speedtest.Speedtest): Speedtest object.
        max_retries (int): Maximum number of retries.
        delay (int): Delay in seconds between retries.
        
    Returns:
        float or None: Download speed in Mbps, or None if measurement failed.
    """
    attempt = 0
    while max_retries == -1 or attempt < max_retries:
        try:
            st.get_best_server()
            download_speed = st.download() / 1024 / 1024  # Convert to Mbps
            return download_speed  # Exit the loop if speed measurement succeeds
        except (speedtest.ConfigRetrievalError, speedtest.SpeedtestTimeout) as e:
            print(f"Error: {e}. Retrying...")
            attempt += 1
            time.sleep(delay)  # Wait before retrying
    return None

def get_plot_size(plot_path):
    """
    Measure the size of a plot file.
    
    Parameters:
        plot_path (str): Path to the plot file.
        
    Returns:
        float: Size of the plot file in MB.
    """
    return os.path.getsize(plot_path) / (1024 * 1024)  # Convert to MB

def calculate_upload_time(file_size, upload_speed):
    """
    Calculate upload time based on file size and upload speed.
    
    Parameters:
        file_size (float): Size of the file in MB.
        upload_speed (float): Upload speed in Mbps.
        
    Returns:
        float: Estimated upload time in seconds.
    """
    return file_size / upload_speed

import os
import time
import speedtest

def measure_speed_with_retry(st, max_retries=3, delay=5):
    """
    Measure internet speed with retries and delay.
    """
    for _ in range(max_retries):
        try:
            return st.download() / 1e6  # Convert to Mbps
        except speedtest.SpeedtestBestServerFailure:
            print("Failed to find the best server. Retrying...")
            time.sleep(delay)
    return None

def get_plot_size(plot_path):
    """
    Get plot file size.
    """
    return os.path.getsize(plot_path) / (1024 * 1024)  # Convert to MB

def calculate_upload_time(plot_size, internet_speed):
    """
    Calculate upload time.
    """
    return plot_size / internet_speed

if __name__ == "__main__":
    while True:
        try:
            plot_path = "G:\Optics_channel\\Plot_Periods\\Output1\\Total_Values_Market_Report.png"

            # Check if the plot file exists
            if os.path.exists(plot_path):
                # Initialize Speedtest object
                st = speedtest.Speedtest()
                st.timeout = 10  # Set a timeout of 60 seconds

                # Measure internet speed with retries and delay
                internet_speed = measure_speed_with_retry(st, max_retries=10, delay=5)

                # Get plot file size
                plot_size = get_plot_size(plot_path)

                # Calculate upload time
                if internet_speed is not None and internet_speed != 0:
                    upload_time = calculate_upload_time(plot_size, internet_speed)
                    print(f"Plot file size: {plot_size:.2f} MB")
                    print(f"Internet speed: {internet_speed:.2f} Mbps")
                    print(f"Estimated upload time: {upload_time:.2f} seconds")
                elif internet_speed == 0:
                    print("Internet speed is zero, cannot calculate upload time.")
                else:
                    print("Failed to measure internet speed.")
            else:
                print("Plot file not found.")
            time.sleep(10)  # Wait for 10 seconds before the next iteration
            break
        except speedtest.ConfigRetrievalError:
            pass

In [ ]:
bot_token = "6846082494:AAEd_6Dg-mOZC4w4y7D_HucEfL3oWD7nnlE"
group_chat_id = '-1002128465774'
#group_chat_id = '988650612' moslem

In [ ]:
max_retries = 2

In [ ]:
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\Last_Price_of_CME_Commodity.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent files
                break
            else:
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again

import os
import time
import speedtest

def measure_speed_with_retry(st, max_retries=3, delay=5):
    """
    Measure internet speed with retries and delay.
    """
    for _ in range(max_retries):
        try:
            return st.download() / 1e6  # Convert to Mbps
        except speedtest.SpeedtestBestServerFailure:
            print("Failed to find the best server. Retrying...")
            time.sleep(delay)
    return None

def get_plot_size(plot_path):
    """
    Get plot file size.
    """
    return os.path.getsize(plot_path) / (1024 * 1024)  # Convert to MB

def calculate_upload_time(plot_size, internet_speed):
    """
    Calculate upload time.
    """
    return plot_size / internet_speed

def calculate_max_retries(plot_size=None, internet_speed=None, upload_time=None):
    """
    Calculate maximum retries.
    """
    if plot_size is None or internet_speed is None or upload_time is None:
        # Default values if any of the parameters is missing
        return 5
    else:
        # Convert Mbps to MB per second
        internet_speed_mbps_to_mbps_per_sec = internet_speed / 8  # 1 byte = 8 bits

        # Calculate the estimated upload time for one attempt in seconds
        estimated_upload_time_per_attempt_sec = plot_size / internet_speed_mbps_to_mbps_per_sec

        # Determine the maximum allowed upload time (e.g., 5 minutes)
        max_allowed_upload_time_sec = 5 * 60  # 5 minutes in seconds

        # Set a minimum estimated upload time (e.g., 1 second) to avoid excessive retries
        min_estimated_upload_time_sec = 1  # 1 second

        # Calculate the maximum number of retries
        max_retries = int(max(min_estimated_upload_time_sec, max_allowed_upload_time_sec) / estimated_upload_time_per_attempt_sec)
        #max_retries = int(estimated_upload_time_per_attempt_sec)
        return int(max_retries/22)

if __name__ == "__main__":
    while True:
        try:
            plot_path = "G:\Optics_channel\\Plot_Periods\\Output1\\Total_Values_Market_Report.png"

            # Check if the plot file exists
            if os.path.exists(plot_path):
                # Initialize Speedtest object
                st = speedtest.Speedtest()
                st.timeout = 10  # Set a timeout of 60 seconds

                # Measure internet speed with retries and delay
                internet_speed = measure_speed_with_retry(st, max_retries=10, delay=5)

                # Get plot file size
                plot_size = get_plot_size(plot_path)

                # Calculate upload time
                upload_time = None  # Default value
                if internet_speed is not None and internet_speed != 0:
                    upload_time = calculate_upload_time(plot_size, internet_speed)
                    print(f"Plot file size: {plot_size:.2f} MB")
                    print(f"Internet speed: {internet_speed:.2f} Mbps")
                    print(f"Estimated upload time: {upload_time:.2f} seconds")
                elif internet_speed == 0:
                    print("Internet speed is zero, cannot calculate upload time.")
                else:
                    print("Failed to measure internet speed.")

                # Calculate maximum retries
                max_retries = calculate_max_retries(plot_size, internet_speed, upload_time)
                print("Maximum retries:", max_retries)

            else:
                print("Plot file not found.")
            time.sleep(10)  # Wait for 10 seconds before the next iteration
            break
        except speedtest.ConfigRetrievalError:
            pass

In [ ]:
max_retries = 10

In [ ]:
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\input_output_money_bors.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent files
                break
            else:
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again

In [ ]:
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\Saf_bors.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent file
                break
            else:
                #print(f"Failed to send photo '{file_path}'.")
                #if max_retries >= 5:
                #    print("Exceeded maximum retry attempts. Exiting...")
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again


In [ ]:
# Initialize the bot
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\Arzesh_bors.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent files
                break
            else:
                print(f"Failed to send photo '{file_path}'.")
                #if max_retries >= 5:
                 #   print("Exceeded maximum retry attempts. Exiting...")
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again


In [ ]:
# Initialize the bot
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\input_output_money_farabors.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent files
                break
            else:
                print(f"Failed to send photo '{file_path}'.")
                #if max_retries >= 5:
                 #   print("Exceeded maximum retry attempts. Exiting...")
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again


In [ ]:
# Initialize the bot
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\Saf_farabors.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent files
                break
            else:
                print(f"Failed to send photo '{file_path}'.")
                #if max_retries >= 5:
                 #   print("Exceeded maximum retry attempts. Exiting...")
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again


In [ ]:
# Initialize the bot
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\Arzesh_farabors.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent files
                break
            else:
                print(f"Failed to send photo '{file_path}'.")
                #if max_retries >= 5:
                 #   print("Exceeded maximum retry attempts. Exiting...")
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again


In [ ]:
# Initialize the bot
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\input_output_money_funds.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent files
                break
            else:
                print(f"Failed to send photo '{file_path}'.")
                #if max_retries >= 5:
                 #   print("Exceeded maximum retry attempts. Exiting...")
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again


In [ ]:
# Initialize the bot
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\Arzesh_funds.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent files
                break
            else:
                print(f"Failed to send photo '{file_path}'.")
                #if max_retries >= 5:
                 #   print("Exceeded maximum retry attempts. Exiting...")
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again


In [ ]:
# Initialize the bot
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\Arzesh_block_bors.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent files
                break
            else:
                print(f"Failed to send photo '{file_path}'.")
                #if max_retries >= 5:
                 #   print("Exceeded maximum retry attempts. Exiting...")
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again


In [ ]:
# Initialize the bot
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\Arzesh_block_farabors.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent files
                break
            else:
                print(f"Failed to send photo '{file_path}'.")
                #if max_retries >= 5:
                 #   print("Exceeded maximum retry attempts. Exiting...")
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again


In [ ]:
# Initialize the bot
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\Arzesh_block_funds.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent files
                break
            else:
                print(f"Failed to send photo '{file_path}'.")
                #if max_retries >= 5:
                 #   print("Exceeded maximum retry attempts. Exiting...")
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again


In [ ]:
# Initialize the bot
bot = Bot(token=bot_token)

# Function to send a photo with a maximum number of retries
async def send_photo_with_retry(bot, chat_id, photo_path, max_retries=10):
    attempts = 0
    while max_retries == -1 or attempts < max_retries:
        try:
            with open(photo_path, 'rb') as photo_file:
                await bot.send_photo(chat_id=chat_id, photo=photo_file)
            print(f'Photo sent to chat ID {chat_id}')
            return True  # Return True if photo is sent successfully
        except Exception as e:
            print(f'Failed to send photo to chat ID {chat_id}. Error: {e}')
            attempts += 1
            if max_retries != -1:
                print(f'Retrying to send photo {photo_path}... (Attempt {attempts}/{max_retries})')
            await asyncio.sleep(5)  # Wait for 5 seconds before retrying
    print(f'Failed to send photo {photo_path} after {max_retries} attempts.')
    return False

# Absolute path to your PNG file
file_path = 'C:\\Optics_faraz_group\\Plot_Periods\\Output8\\Total_market_value.png'

# Set to keep track of successfully sent plots
successfully_sent_plots = set()

# Infinite loop to continuously process the plot file
while True:
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f'File not found: {file_path}')
    else:
        # Attempt to send the plot file with a maximum of 10 retries
        if file_path not in successfully_sent_plots:
            print(f'Sending photo: {file_path}')
            if await send_photo_with_retry(bot, group_chat_id, file_path, max_retries= max_retries):
                print(f"Photo '{file_path}' sent successfully.")
                successfully_sent_plots.add(file_path)  # Add file to set of successfully sent files
                break
            else:
                print(f"Failed to send photo '{file_path}'.")
                #if max_retries >= 5:
                 #   print("Exceeded maximum retry attempts. Exiting...")
                break
        else:
            print(f"Photo '{file_path}' has already been sent. Skipping...")
            break
    # Wait for some time before processing the file again
    await asyncio.sleep(60)  # Wait for 60 seconds before processing the file again


In [ ]:
import os

current_directory = os.getcwd()
print("Current script directory:", current_directory)